# Pytorch_llm_intro_ja.ipynb

こちらは日本語向けのLLM(Large Language Model)のチュートリアルです。<br>
ここでは、[rinna株式会社](https://corp.rinna.co.jp/)によって学習された、日本語のGPTモデルを使います。<br>
これらのモデルは、[The MIT license](https://opensource.org/license/mit/)で提供されています。

このチュートリアルは、Google Colaboratoryで、GPUを使って実行されることを想定しています。

This is the tutorial for LLM(Large Language Model) for Japanese.<br>
We use Japanese GPT model trained by [rinna Co., Ltd.](https://corp.rinna.co.jp/).<br>
These models are under [The MIT license](https://opensource.org/license/mit/).

I assume that this tutorial is used on Google Colaboratory with GPU.

-----

https://huggingface.co/rinna/japanese-gpt-neox-small <br>
https://huggingface.co/rinna/japanese-gpt-neox-3.6b-instruction-ppo

これらが、今回使用したモデルのドキュメントとなっております。

These are the pages of documents for the models that we use this time.

## ライブラリをpip installする

ここでは、必要なライブラリをpipを用いてインストールします。

On this part we will pip install library that we need.

-----

https://www.cloudnotes.tech/entry/rinna_gpt_jp

こちらのページによると、もしうまく動かなかったらカーネルをリスタートする必要があるとのことです。

According to this page, we have to restart kernel if SentencePiece doesn't work well.

In [ ]:
! pip install transformers
! pip install sentencepiece
! pip install accelerate

## ライブラリをインポートする

ここでは、必要なライブラリをインポートします。
- Pytorch
- transformers

PytorchはGoogle Colaboratoryにはデフォルトでインストールされています。<br>
[transformers](https://github.com/huggingface/transformers)は、[HuggingFace社](https://huggingface.co/)が開発を行っている機械学習向けのライブラリです。

もし現時点でGPUが使えない状態の場合は、`False`が出力されます。<br>
もし`False`が出力されたら、GPUを使えるよう変更してください。


Here we import libraries that we need.
- Pytorch
- transformers

Pytorch is pre-installed on Google Colaboratory.<br>
[transformers](https://github.com/huggingface/transformers) is library for machine learning that [HuggingFace, Inc.](https://huggingface.co/) develops.

If you can't use GPU now, this notebook outputs `False`.<br>
If model outputs `False`, you should change setting to use GPU.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print(torch.cuda.is_available())

## 話の続きを予測する

まずは、話の続きを予測するモデルを試してみましょう。

For first, we use model that predicts continuation of the story.

### TokenizerとModelを定義する

ファイルを読み込んで、`tokenizer`と`model`を定義します。

Read files and define `tokenizer` and `model`.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-small", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-small")

# もしcuda(GPU)が使用可能であれば、モデルをGPUに移します。
# If cuda(GPU) is available, we throw model to GPU.
if torch.cuda.is_available():
    model = model.to("cuda")

### Textを定義する

`text`を定義します。<br>
ここで`text`とは、続きを予測させる文章の出だしを表します。<br>
**ここでは内容を自由に変更することができます。**

Define `text`.<br>
`text`, that we use for now, means the sentences whose continuation that we want.<br>
**Here you can change codes freely.**

In [ ]:
text = "東京工業大学とは、"

## 返答を予測する

ここでは、モデルが続きを予測します。
少し時間が掛かります。

Here model predicts continuation.
This part takes some times.

In [ ]:
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=100,
        min_new_tokens=100,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

## 会話の続きを予測する

次に、会話の続きを予測するモデルを試してみましょう。

Next, we use the model that predicts continuation of dialogue.

### TokenizerとModelを定義する

ファイルを読み込んで、`tokenizer`と`model`を定義します。<br>
**モデルが大きいので、実行される際はストレージやRAMにご注意ください。**

Read files and define `tokenizer` and `model`.<br>
**Please pay attention to free space of your storage or RAM because the model is too big if you run this section.**

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-ppo", use_fast=False)
model2 = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-neox-3.6b-instruction-ppo", device_map="auto")

### Promptを定義する

`prompt`を定義します。<br>
ここで`prompt`とは、言語モデルに渡す会話のことを表します。<br>
**ここでは内容を自由に変更することができます。**

Define `prompt`.<br>
`prompt`, that we use for now, means conversation that we pass to language model.<br>
**Here You can change codes freely.**

In [ ]:
prompt = [
    {
        "speaker": "ユーザー",
        "text": "昔々あるところに、"
    },
]

## Promptを処理する

先ほどのパートで定義した`prompt`を処理します。

Here `prompt` that you defined last part is processed.

In [ ]:
prompt = [
    f"{uttr['speaker']}: {uttr['text']}"
    for uttr in prompt
]
prompt = "<NL>".join(prompt)
prompt = (
    prompt
    + "<NL>"
    + "システム: "
)
print(prompt)

## 返答を予測する

ここでは、モデルが返答を予測します。

Here model predicts reply.

In [ ]:
token_ids2 = tokenizer2.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids2 = model2.generate(
        token_ids2.to(model2.device),
        do_sample=True,
        max_new_tokens=128,
        temperature=0.7,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

output2 = tokenizer2.decode(output_ids2.tolist()[0][token_ids2.size(1):])
output2 = output2.replace("<NL>", "\n")
print(output2)